<a href="https://colab.research.google.com/github/SarahVSargent/stock_signals_regressions/blob/main/ERTIMUR_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The below is the standard set of imports according to Powley

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Import tools
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

!pip install pandasql

import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats.mstats import winsorize
import math
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import pandasql as ps

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=5a88057b741348ba73adbb6f7bf350e0d26893729d7f7950c749877fa1fb2be4
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


Importing files

In [ ]:
file = "/content/gdrive/MyDrive/FIMA/Compustat2018Through2022.csv"
Compustat2018Through2022 = pd.read_csv(file)
print(Compustat2018Through2022.columns)

file = "/content/gdrive/MyDrive/FIMA/CRSP2020Through2022.csv"
CRSP2020Through2022 = pd.read_csv(file)

print("imported")

<ipython-input-3-1e5c29c739d0>:2: DtypeWarning: Columns (17,18,26,30,647,669,674) have mixed types. Specify dtype option on import or set low_memory=False.
  Compustat2018Through2022 = pd.read_csv(file)


Index(['gvkey', 'datadate', 'fyearq', 'fqtr', 'fyr', 'indfmt', 'consol',
       'popsrc', 'datafmt', 'tic',
       ...
       'priusa', 'sic', 'spcindcd', 'spcseccd', 'spcsrc', 'state', 'stko',
       'weburl', 'dldte', 'ipodate'],
      dtype='object', length=680)


<ipython-input-3-1e5c29c739d0>:6: DtypeWarning: Columns (5,9,18,49,57) have mixed types. Specify dtype option on import or set low_memory=False.
  CRSP2020Through2022 = pd.read_csv(file)


imported


Cleaning Compustat file (6:26)

In [ ]:
Compustat2018Through2022["datadate"] = pd.to_datetime(Compustat2018Through2022["datadate"], format="%Y%m%d")
Compustat2018Through2022["rdq"] = pd.to_datetime(Compustat2018Through2022["rdq"], format="%Y%m%d")
Compustat2018Through2022["cshoq"] = pd.to_numeric(Compustat2018Through2022["cshoq"], errors="coerce")
Compustat2018Through2022["ceqq"] = pd.to_numeric(Compustat2018Through2022["ceqq"], errors="coerce")
Compustat2018Through2022["ibq"] = pd.to_numeric(Compustat2018Through2022["ibq"], errors="coerce")
Compustat2018Through2022["saleq"] = pd.to_numeric(Compustat2018Through2022["saleq"], errors="coerce")
Compustat2018Through2022["prccq"] = pd.to_numeric(Compustat2018Through2022["prccq"], errors="coerce")
print("clean 1")

#indfmt = industry format (char)
#datafmt = data format (char)
Compustat2018Through2022["indfmt"] = Compustat2018Through2022["indfmt"].apply(str)
Compustat2018Through2022["datafmt"] = Compustat2018Through2022["datafmt"].apply(str)
print("ind/datafmt")

#in table compustat, make sure that only correct industry and data format is used
#indfmt = INDL; datafmt = "STD"
Compustat2018Through2022 = Compustat2018Through2022[Compustat2018Through2022["indfmt"] == "INDL"]
Compustat2018Through2022 = Compustat2018Through2022[Compustat2018Through2022["datafmt"] == "STD"]
print("proper format")

#only take dates after 2020
from sqlite3 import connect
conn = connect(":memory:")

print("conn")

Compustat2018Through2022.to_sql("Compustat2018Through2022", conn, if_exists="replace")
query = '''SELECT DISTINCT *
        FROM Compustat2018Through2022
        WHERE Compustat2018Through2022.datadate >= "2020-01-01"
        '''
Compustat2020Through2022 = pd.read_sql(query, conn)

print("after 2020")

print(Compustat2020Through2022.head(10))

clean 1
ind/datafmt
proper format
conn
after 2020
   index  gvkey             datadate  fyearq  fqtr  fyr indfmt consol popsrc  \
0      8   1004  2020-02-29 00:00:00    2019   3.0    5   INDL      C      D   
1      9   1004  2020-05-31 00:00:00    2019   4.0    5   INDL      C      D   
2     10   1004  2020-08-31 00:00:00    2020   1.0    5   INDL      C      D   
3     11   1004  2020-11-30 00:00:00    2020   2.0    5   INDL      C      D   
4     12   1004  2021-02-28 00:00:00    2020   3.0    5   INDL      C      D   
5     13   1004  2021-05-31 00:00:00    2020   4.0    5   INDL      C      D   
6     14   1004  2021-08-31 00:00:00    2021   1.0    5   INDL      C      D   
7     15   1004  2021-11-30 00:00:00    2021   2.0    5   INDL      C      D   
8     16   1004  2022-02-28 00:00:00    2021   3.0    5   INDL      C      D   
9     25   1045  2020-03-31 00:00:00    2020   1.0   12   INDL      C      D   

  datafmt  ... priusa     sic spcindcd spcseccd spcsrc  state  stko  

Cleaning CRSP; getting tickers and earnings dates from Compustat (6:47)

In [ ]:
CRSP2020Through2022["date"] = pd.to_datetime(CRSP2020Through2022["date"], format="%Y%m%d")

print("crsp dates")

Compustat2020Through2022.to_sql("Compustat2020Through2022", conn, if_exists="replace")
query = '''SELECT DISTINCT Compustat2020Through2022.tic, Compustat2020Through2022.rdq
        FROM Compustat2020Through2022
        WHERE Compustat2020Through2022.tic <> "" and Compustat2020Through2022.rdq <> ""
        '''
Compustat2020Through2022UniqueGVKeysAndRDQs = pd.read_sql(query, conn)

print("unique gvs and rdqs")

crsp dates
unique gvs and rdqs


Get tickers and dates from CRSP (7:38)

In [ ]:
CRSP2020Through2022.to_sql("CRSP2020Through2022", conn, if_exists="replace")
query = '''SELECT DISTINCT CRSP2020Through2022.TICKER, CRSP2020Through2022.date
        FROM CRSP2020Through2022
        '''

CRSP2020Through2022UniqueTicsAndDates = pd.read_sql(query, conn)

print("unique tics")

unique tics


Left join tickers and dates from CRSP with earnings dates from Compustat (10:03)

In [ ]:
CRSP2020Through2022UniqueTicsAndDates.to_sql("CRSP2020Through2022UniqueTicsAndDates", conn, if_exists="replace")
Compustat2020Through2022UniqueGVKeysAndRDQs.to_sql("Compustat2020Through2022UniqueGVKeysAndRDQs", conn, if_exists="replace")
query = '''SELECT DISTINCT CRSP2020Through2022UniqueTicsAndDates.TICKER, CRSP2020Through2022UniqueTicsAndDates.date, Compustat2020Through2022UniqueGVKeysAndRDQs.rdq
        FROM CRSP2020Through2022UniqueTicsAndDates
        LEFT JOIN Compustat2020Through2022UniqueGVKeysAndRDQs
        ON CRSP2020Through2022UniqueTicsAndDates.TICKER = Compustat2020Through2022UniqueGVKeysAndRDQs.tic
        and CRSP2020Through2022UniqueTicsAndDates.date = Compustat2020Through2022UniqueGVKeysAndRDQs.rdq
        ORDER BY CRSP2020Through2022UniqueTicsAndDates.TICKER, CRSP2020Through2022UniqueTicsAndDates.date
        '''
CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ = pd.read_sql(query, conn)
print(CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.head(30))

   TICKER                 date   rdq
0    None  2020-01-02 00:00:00  None
1    None  2020-01-03 00:00:00  None
2    None  2020-01-06 00:00:00  None
3    None  2020-01-07 00:00:00  None
4    None  2020-01-08 00:00:00  None
5    None  2020-01-09 00:00:00  None
6    None  2020-01-10 00:00:00  None
7    None  2020-01-13 00:00:00  None
8    None  2020-01-14 00:00:00  None
9    None  2020-01-15 00:00:00  None
10   None  2020-01-16 00:00:00  None
11   None  2020-01-17 00:00:00  None
12   None  2020-01-21 00:00:00  None
13   None  2020-01-22 00:00:00  None
14   None  2020-01-23 00:00:00  None
15   None  2020-01-24 00:00:00  None
16   None  2020-01-27 00:00:00  None
17   None  2020-01-28 00:00:00  None
18   None  2020-01-29 00:00:00  None
19   None  2020-01-30 00:00:00  None
20   None  2020-01-31 00:00:00  None
21   None  2020-02-03 00:00:00  None
22   None  2020-02-04 00:00:00  None
23   None  2020-02-05 00:00:00  None
24   None  2020-02-06 00:00:00  None
25   None  2020-02-07 00:00:00  None
2

Keep only the rows three days after the earnings date (13:39)

In [ ]:
CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.sort_values(by=["TICKER", "date"], ascending=False)

#Calculating three day return window
CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ["rdq_plus1"] = \
    CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.groupby(["TICKER"])["rdq"].shift(1)
CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ["rdq_plus2"] = \
    CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.groupby(["TICKER"])["rdq"].shift(2)
CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ["rdq_plus3"] = \
    CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.groupby(["TICKER"])["rdq"].shift(3)

#TODO: Ask FIMA team abt this
CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ["three_days_after_earnings_if_intraday"] \
    = CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ["rdq_plus2"]
CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ["three_days_after_earnings_if_afterhours"] \
    = CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ["rdq_plus3"]

CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.to_sql("CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ", conn, if_exists="replace")
query = '''SELECT DISTINCT CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.TICKER, CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.date,
        CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.rdq, CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.rdq_plus1,
        CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.rdq_plus2, CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.rdq_plus3
        FROM CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ
        WHERE CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.rdq <> "NaN" or CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.rdq_plus1 <> "NaN"
        or CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.rdq_plus2 <> "NaN" or CRSPCompustat2020Through2022UniqueTicsAndDatesAndRDQ.rdq_plus3 <> "NaN"
        '''
CRSPCompustat2020Through2022ThreeDaysFollowingRDQ = pd.read_sql(query, conn)

Add in return data (17:00)

In [ ]:
CRSPCompustat2020Through2022ThreeDaysFollowingRDQ.to_sql("CRSPCompustat2020Through2022ThreeDaysFollowingRDQ", conn, if_exists="replace")
CRSP2020Through2022.to_sql("CRSP2020Through2022", conn, if_exists="replace")
query = '''SELECT DISTINCT CRSPCompustat2020Through2022ThreeDaysFollowingRDQ.TICKER, CRSPCompustat2020Through2022ThreeDaysFollowingRDQ.date,
        CRSPCompustat2020Through2022ThreeDaysFollowingRDQ.rdq, CRSP2020Through2022.RET
        FROM CRSPCompustat2020Through2022ThreeDaysFollowingRDQ
        LEFT JOIN CRSP2020Through2022
        ON CRSPCompustat2020Through2022ThreeDaysFollowingRDQ.TICKER = CRSP2020Through2022.TICKER and
        CRSPCompustat2020Through2022ThreeDaysFollowingRDQ.date = CRSP2020Through2022.date
        '''
CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturns = pd.read_sql(query, conn)

Add cash dividend payout YTD, sales, net income, shareholder equity, shares outstanding, price (20:34)

In [ ]:
CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturns.to_sql("CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturns", conn, if_exists="replace")
Compustat2020Through2022.to_sql("Compustat2020Through2022", conn, if_exists="replace")
query = '''SELECT DISTINCT CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturns.TICKER, CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturns.date,
        CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturns.rdq, CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturns.RET,
        Compustat2020Through2022.datadate, Compustat2020Through2022.dvy, Compustat2020Through2022.saleq, Compustat2020Through2022.ibq,
        Compustat2020Through2022.fyearq, Compustat2020Through2022.fqtr, Compustat2020Through2022.fyr, Compustat2020Through2022.ceqq, Compustat2020Through2022.cshoq,
        Compustat2020Through2022.prccq
        FROM CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturns
        LEFT JOIN Compustat2020Through2022
        ON CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturns.TICKER = Compustat2020Through2022.tic and
        CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturns.rdq = Compustat2020Through2022.rdq
        '''
CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturnsDataDateSaleqNiqFyearqFyrFqtr = pd.read_sql(query, conn)

Add in cumulative three-day return (23:37)

In [ ]:
table = CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturnsDataDateSaleqNiqFyearqFyrFqtr
table["ret_1"] = table.groupby(["TICKER"])["RET"].shift(-1)
table["ret_2"] = table.groupby(["TICKER"])["RET"].shift(-2)
table["ret_3"] = table.groupby(["TICKER"])["RET"].shift(-3)
table["RET"] = pd.to_numeric(table["RET"], errors="coerce")
table["ret_1"] = pd.to_numeric(table["ret_1"], errors="coerce")
table["ret_2"] = pd.to_numeric(table["ret_2"], errors="coerce")
table["ret_3"] = pd.to_numeric(table["ret_3"], errors="coerce")
#TODO: ask about three day return if intraday. timestamp 31:11.
table["three_day_return_if_intraday"] = (1 + table["RET"]) * (1 + table["ret_1"]) * (1 + table["ret_2"]) - 1
table["three_day_return_if_afterhours"] = (1 + table["ret_1"]) * (1 + table["ret_2"]) * (1 + table["ret_3"]) - 1
CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturnsDataDateSaleqNiqFyearqFyrFqtr = table

Keep only the rows with an earnings announcement (24:06)

In [ ]:
#here i will just start using pandas
CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturnsDataDateSaleqNiqFyearqFyrFqtrNonNullRDQ = \
    CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturnsDataDateSaleqNiqFyearqFyrFqtr \
        [CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturnsDataDateSaleqNiqFyearqFyrFqtr["date"] == \
            CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturnsDataDateSaleqNiqFyearqFyrFqtr["rdq"]]

Create a table with 2019 data and the same rows (27:23)

In [ ]:
cols = ["tic", "rdq", "datadate", "dvy", "saleq", "ibq", "fyearq", "fqtr", "fyr", "ceqq", "cshoq", "prccq"]
compustat2019allcols = Compustat2018Through2022.loc[(Compustat2018Through2022["fyearq"] == 2019), cols]
compustat2019allcols.rename(columns = {"tic" : "TICKER"}, inplace=True)
compustat2019allcols.sort_values(by=["TICKER", "datadate"])

,TICKER,rdq,datadate,dvy,saleq,ibq,fyearq,fqtr,fyr,ceqq,cshoq,prccq
122892,0048A,2019-04-26,2019-03-31,3.970,836.549,21.835,2019,1.0,12,1580.704,24.500,NaN
122893,0048A,2019-07-31,2019-06-30,5.208,878.937,79.108,2019,2.0,12,NaN,24.500,NaN
122894,0048A,2019-10-25,2019-09-30,37.097,888.475,12.425,2019,3.0,12,NaN,24.500,NaN
122895,0048A,2020-02-13,2019-12-31,58.347,938.125,60.658,2019,4.0,12,1662.324,26.620,NaN
1960,0141A,2019-05-15,2019-03-31,0.000,886.000,-52.000,2019,1.0,12,-2964.000,82.557,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
153230,NaN,2019-11-25,2019-09-30,0.000,0.288,-0.375,2019,3.0,12,4.652,48.313,0.190
74416,NaN,2020-02-26,2019-12-31,0.000,0.000,-1.465,2019,4.0,12,2.075,70.946,0.345
133712,NaN,2020-02-17,2019-12-31,470.000,938.000,157.000,2019,4.0,12,5009.000,498.668,NaN
153231,NaN,2020-04-29,2019-12-31,0.000,0.424,-0.313,2019,4.0,12,4.495,49.190,0.230


Prepend Compustat 2019 File to Main Table (29:00)

In [ ]:
table2019to2022 = compustat2019allcols.append(CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturnsDataDateSaleqNiqFyearqFyrFqtrNonNullRDQ)
#subset_cols = [] #need subset
#table2019to2022.drop_duplicates(inplace=True)
table2019to2022.sort_values(by=["TICKER", "datadate"], ascending=False)

<ipython-input-14-9bce04afaf15>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table2019to2022 = compustat2019allcols.append(CRSPCompustat2020Through2022ThreeDaysFollowingRDQwithReturnsDataDateSaleqNiqFyearqFyrFqtrNonNullRDQ)
<ipython-input-14-9bce04afaf15>:4: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  table2019to2022.sort_values(by=["TICKER", "datadate"], ascending=False)


,TICKER,rdq,datadate,dvy,saleq,ibq,fyearq,fqtr,fyr,ceqq,cshoq,prccq,date,RET,ret_1,ret_2,ret_3,three_day_return_if_intraday,three_day_return_if_afterhours
163530,ZZZOF,2020-10-21 00:00:00,2020-02-29 00:00:00,0.0,0.000,-6.399,2019.0,4.0,2.0,-2.006,193.957,0.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163529,ZZZOF,2020-01-29 00:00:00,2019-11-30 00:00:00,0.0,0.000,-0.184,2019.0,3.0,2.0,4.321,193.957,0.020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163528,ZZZOF,2019-10-30 00:00:00,2019-08-31 00:00:00,0.0,0.000,-0.415,2019.0,2.0,2.0,4.514,193.957,0.020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163527,ZZZOF,2019-07-30 00:00:00,2019-05-31 00:00:00,0.0,0.000,-0.687,2019.0,1.0,2.0,3.690,122.361,0.045,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89205,ZZZD.,NaT,2019-12-31 00:00:00,NaN,NaN,NaN,2019.0,4.0,12.0,NaN,1.940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133710,NaN,2019-08-04 00:00:00,2019-06-30 00:00:00,229.0,929.000,121.000,2019.0,2.0,12.0,4946.000,498.668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153229,NaN,2019-08-22 00:00:00,2019-06-30 00:00:00,0.0,0.265,-0.376,2019.0,2.0,12.0,3.991,44.670,0.160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74413,NaN,2019-05-22 00:00:00,2019-03-31 00:00:00,0.0,0.000,-0.401,2019.0,1.0,12.0,1.233,57.788,0.280,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133709,NaN,2019-05-09 00:00:00,2019-03-31 00:00:00,140.0,902.000,167.000,2019.0,1.0,12.0,4873.000,498.668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Add MV of eq and BTM (30:34)

In [ ]:
table2019to2022["market_value_of_equity"] = table2019to2022["cshoq"] * table2019to2022["prccq"]
table2019to2022["book_to_market_timet"] = table2019to2022["ceqq"] / (table2019to2022["cshoq"] * table2019to2022["prccq"])

Rank BTM at each FQ and delinate first third from last third

In [ ]:
#If BTM is negative, set it equal to 0.5
table2019to2022["book_to_market_timet"] = table2019to2022["book_to_market_timet"].apply(lambda x: 0.5 if x < 0 else x)
#create new BTM data df
BTM2019through2022 = table2019to2022[['TICKER', 'datadate', 'book_to_market_timet']].copy()
#If BTM is null, scrap it.
BTM2019through2022.dropna(subset=["book_to_market_timet"], inplace=True)
#create new columns, set default to False
BTM2019through2022['FirstThird'] = False
BTM2019through2022['LastThird'] = False
#assign percentiles
BTM2019through2022["percentile"] = BTM2019through2022.groupby(["datadate"])["book_to_market_timet"].rank(pct=True)
BTM2019through2022.loc[BTM2019through2022["percentile"] <= .33, "FirstThird"] = True
BTM2019through2022.loc[BTM2019through2022["percentile"] >= .67, "LastThird"] = True

Rank YTD div yield for all companies at each FQ, delineate first/last qtls. (35:54)

In [ ]:
#create new div yield data df
divyielddata2019through2022 = table2019to2022[["TICKER", "datadate", "dvy", "market_value_of_equity"]].copy()
# Calculating dividendyield
divyielddata2019through2022["dividendyield"] = divyielddata2019through2022["dvy"] / divyielddata2019through2022["market_value_of_equity"]
#drop null market value of equity
divyielddata2019through2022.dropna(subset=["market_value_of_equity"], inplace=True)
#drop unneeded columns
divyielddata2019through2022.drop(["market_value_of_equity", "dvy"], axis=1, inplace=True)
#create new columns, set default to False
divyielddata2019through2022['FirstQuartile'] = False
divyielddata2019through2022['FourthQuartile'] = False
#assign percentiles
divyielddata2019through2022["percentile"] = divyielddata2019through2022.groupby(["datadate"])["dividendyield"].rank(pct=True)
divyielddata2019through2022.loc[divyielddata2019through2022["percentile"] <= .25, "FirstQuartile"] = True
divyielddata2019through2022.loc[divyielddata2019through2022["percentile"] >= .75, "FourthQuartile"] = True

Merge BTM and div yields into main table (36:41)

In [ ]:
table2019to2022 = table2019to2022.merge(
    BTM2019through2022.rename(columns={'FirstThird': 'btmfirstthird', 'LastThird': 'btmlastthird'})[['TICKER', 'datadate', 'btmfirstthird', 'btmlastthird']],
    on=['TICKER', 'datadate'],
    how='left'
)

table2019to2022 = table2019to2022.merge(
    divyielddata2019through2022.rename(columns={'FirstQuartile': 'dvyfirstquartile', 'FourthQuartile': 'dvyfourthquartile'})[['TICKER', 'datadate', 'dvyfirstquartile', 'dvyfourthquartile']],
    on=['TICKER', 'datadate'],
    how='left'
)
table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending=False)

<ipython-input-18-c137e2bbd23f>:12: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending=False)


,TICKER,rdq,datadate,dvy,saleq,ibq,fyearq,fqtr,fyr,ceqq,...,ret_2,ret_3,three_day_return_if_intraday,three_day_return_if_afterhours,market_value_of_equity,book_to_market_timet,btmfirstthird,btmlastthird,dvyfirstquartile,dvyfourthquartile
39096,ZZZOF,2020-10-21 00:00:00,2020-02-29 00:00:00,0.0,0.000,-6.399,2019.0,4.0,2.0,-2.006,...,NaN,NaN,NaN,NaN,2.909355,0.500000,False,False,False,False
39095,ZZZOF,2020-01-29 00:00:00,2019-11-30 00:00:00,0.0,0.000,-0.184,2019.0,3.0,2.0,4.321,...,NaN,NaN,NaN,NaN,3.879140,1.113907,False,True,False,False
39094,ZZZOF,2019-10-30 00:00:00,2019-08-31 00:00:00,0.0,0.000,-0.415,2019.0,2.0,2.0,4.514,...,NaN,NaN,NaN,NaN,3.879140,1.163660,False,True,False,False
39093,ZZZOF,2019-07-30 00:00:00,2019-05-31 00:00:00,0.0,0.000,-0.687,2019.0,1.0,2.0,3.690,...,NaN,NaN,NaN,NaN,5.506245,0.670148,False,True,False,False
22147,ZZZD.,NaT,2019-12-31 00:00:00,NaN,NaN,NaN,2019.0,4.0,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31834,NaN,2019-05-09 00:00:00,2019-03-31 00:00:00,140.0,902.000,167.000,2019.0,1.0,12.0,4873.000,...,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False
36580,NaN,2019-05-24 00:00:00,2019-03-31 00:00:00,0.0,0.113,-0.342,2019.0,1.0,12.0,1.133,...,NaN,NaN,NaN,NaN,6.340110,0.178704,True,False,False,False
36581,NaN,2019-05-24 00:00:00,2019-03-31 00:00:00,0.0,0.113,-0.342,2019.0,1.0,12.0,1.133,...,NaN,NaN,NaN,NaN,6.340110,0.178704,True,False,False,False
36582,NaN,2019-05-24 00:00:00,2019-03-31 00:00:00,0.0,0.113,-0.342,2019.0,1.0,12.0,1.133,...,NaN,NaN,NaN,NaN,6.340110,0.178704,True,False,False,False


Add year ago quarterly sales and income (38:35)

In [ ]:
table2019to2022['saleq_4'] = table2019to2022.groupby(['TICKER'])['saleq'].shift(4)
table2019to2022['ibq_4'] = table2019to2022.groupby(['TICKER'])['ibq'].shift(4)
table2019to2022['fqtr_4'] = table2019to2022.groupby(['TICKER'])['fqtr'].shift(4)
table2019to2022['fyearq_4'] = table2019to2022.groupby(['TICKER'])['fyearq'].shift(4)

for row in table2019to2022.index:
    try:
        if (table2019to2022['fyearq'][row] - 1 != table2019to2022['fyearq_4'][row]) \
            or (table2019to2022['fqtr'][row] != table2019to2022['fqtr_4'][row]):
            table2019to2022.at[row, 'saleq_4'] = np.nan
            table2019to2022.at[row, 'ibq_4'] = np.nan
            #print(row)
    except:
        continue

table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending=False)

<ipython-input-19-14755132cdf8>:16: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending=False)


,TICKER,rdq,datadate,dvy,saleq,ibq,fyearq,fqtr,fyr,ceqq,...,market_value_of_equity,book_to_market_timet,btmfirstthird,btmlastthird,dvyfirstquartile,dvyfourthquartile,saleq_4,ibq_4,fqtr_4,fyearq_4
39096,ZZZOF,2020-10-21 00:00:00,2020-02-29 00:00:00,0.0,0.000,-6.399,2019.0,4.0,2.0,-2.006,...,2.909355,0.500000,False,False,False,False,NaN,NaN,NaN,NaN
39095,ZZZOF,2020-01-29 00:00:00,2019-11-30 00:00:00,0.0,0.000,-0.184,2019.0,3.0,2.0,4.321,...,3.879140,1.113907,False,True,False,False,NaN,NaN,NaN,NaN
39094,ZZZOF,2019-10-30 00:00:00,2019-08-31 00:00:00,0.0,0.000,-0.415,2019.0,2.0,2.0,4.514,...,3.879140,1.163660,False,True,False,False,NaN,NaN,NaN,NaN
39093,ZZZOF,2019-07-30 00:00:00,2019-05-31 00:00:00,0.0,0.000,-0.687,2019.0,1.0,2.0,3.690,...,5.506245,0.670148,False,True,False,False,NaN,NaN,NaN,NaN
22147,ZZZD.,NaT,2019-12-31 00:00:00,NaN,NaN,NaN,2019.0,4.0,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31834,NaN,2019-05-09 00:00:00,2019-03-31 00:00:00,140.0,902.000,167.000,2019.0,1.0,12.0,4873.000,...,NaN,NaN,True,False,False,False,NaN,NaN,NaN,NaN
36580,NaN,2019-05-24 00:00:00,2019-03-31 00:00:00,0.0,0.113,-0.342,2019.0,1.0,12.0,1.133,...,6.340110,0.178704,True,False,False,False,NaN,NaN,NaN,NaN
36581,NaN,2019-05-24 00:00:00,2019-03-31 00:00:00,0.0,0.113,-0.342,2019.0,1.0,12.0,1.133,...,6.340110,0.178704,True,False,False,False,NaN,NaN,NaN,NaN
36582,NaN,2019-05-24 00:00:00,2019-03-31 00:00:00,0.0,0.113,-0.342,2019.0,1.0,12.0,1.133,...,6.340110,0.178704,True,False,False,False,NaN,NaN,NaN,NaN


Add quarter ago sales and income (40:38)

In [ ]:
table2019to2022['saleq_1'] = table2019to2022.groupby(['TICKER'])['saleq'].shift(1)
table2019to2022['ibq_1'] = table2019to2022.groupby(['TICKER'])['ibq'].shift(1)
table2019to2022['fqtr_1'] = table2019to2022.groupby(['TICKER'])['fqtr'].shift(1)
table2019to2022['fyearq_1'] = table2019to2022.groupby(['TICKER'])['fyearq'].shift(1)

for row in table2019to2022.index:
    try:
        #fq1 case
        if ((table2019to2022['fqtr'][row] == 1.0) and (table2019to2022['fyearq'][row] - 1 != table2019to2022['fyearq_1'][row])) \
            or ((table2019to2022['fqtr'][row] == 1.0) and (table2019to2022['fqtr_1'][row] != 4.0)):
            table2019to2022.at[row, 'fqtr_1'] = np.nan
            table2019to2022.at[row, 'fyearq_1'] = np.nan
    except:
        continue
    try:
        #other cases
        if ((table2019to2022['fqtr'][row] > 1.0) and (table2019to2022['fyearq'][row] != table2019to2022['fyearq_1'][row])) \
            or ((table2019to2022['fqtr'][row] > 1.0) and (table2019to2022['fqtr_1'][row] != table2019to2022['fqtr'][row] - 1)):
            table2019to2022.at[row, 'fqtr_1'] = np.nan
            table2019to2022.at[row, 'fyearq_1'] = np.nan
    except:
        continue

table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending=False)

<ipython-input-20-5658958382c8>:24: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending=False)


,TICKER,rdq,datadate,dvy,saleq,ibq,fyearq,fqtr,fyr,ceqq,...,dvyfirstquartile,dvyfourthquartile,saleq_4,ibq_4,fqtr_4,fyearq_4,saleq_1,ibq_1,fqtr_1,fyearq_1
39096,ZZZOF,2020-10-21 00:00:00,2020-02-29 00:00:00,0.0,0.000,-6.399,2019.0,4.0,2.0,-2.006,...,False,False,NaN,NaN,NaN,NaN,0.0,-0.184,3.0,2019.0
39095,ZZZOF,2020-01-29 00:00:00,2019-11-30 00:00:00,0.0,0.000,-0.184,2019.0,3.0,2.0,4.321,...,False,False,NaN,NaN,NaN,NaN,0.0,-0.415,2.0,2019.0
39094,ZZZOF,2019-10-30 00:00:00,2019-08-31 00:00:00,0.0,0.000,-0.415,2019.0,2.0,2.0,4.514,...,False,False,NaN,NaN,NaN,NaN,0.0,-0.687,1.0,2019.0
39093,ZZZOF,2019-07-30 00:00:00,2019-05-31 00:00:00,0.0,0.000,-0.687,2019.0,1.0,2.0,3.690,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22147,ZZZD.,NaT,2019-12-31 00:00:00,NaN,NaN,NaN,2019.0,4.0,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31834,NaN,2019-05-09 00:00:00,2019-03-31 00:00:00,140.0,902.000,167.000,2019.0,1.0,12.0,4873.000,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36580,NaN,2019-05-24 00:00:00,2019-03-31 00:00:00,0.0,0.113,-0.342,2019.0,1.0,12.0,1.133,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36581,NaN,2019-05-24 00:00:00,2019-03-31 00:00:00,0.0,0.113,-0.342,2019.0,1.0,12.0,1.133,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36582,NaN,2019-05-24 00:00:00,2019-03-31 00:00:00,0.0,0.113,-0.342,2019.0,1.0,12.0,1.133,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Drop all rows with blank year ago and quarter ago sales and income (43:13)

In [ ]:
table2019to2022.dropna(subset=['rdq', 'saleq_4', 'ibq_4', 'saleq_1', 'ibq_1'], inplace=True)
print(table2019to2022)

      TICKER                  rdq             datadate      dvy     saleq  \
45892      A  2020-02-18 00:00:00  2020-01-31 00:00:00   56.000  1357.000   
45893      A  2020-05-21 00:00:00  2020-04-30 00:00:00  111.000  1238.000   
45894      A  2020-08-18 00:00:00  2020-07-31 00:00:00  167.000  1261.000   
45895      A  2020-11-23 00:00:00  2020-10-31 00:00:00  222.000  1483.000   
45896      A  2021-02-16 00:00:00  2021-01-31 00:00:00   59.000  1548.000   
...      ...                  ...                  ...      ...       ...   
85954   ZYXI  2021-02-25 00:00:00  2020-12-31 00:00:00    0.000    25.605   
85955   ZYXI  2021-04-29 00:00:00  2021-03-31 00:00:00    0.000    24.127   
85956   ZYXI  2021-07-29 00:00:00  2021-06-30 00:00:00    0.000    31.022   
85957   ZYXI  2021-11-02 00:00:00  2021-09-30 00:00:00    0.000    34.786   
85958   ZYXI  2022-02-24 00:00:00  2021-12-31 00:00:00    0.001    40.366   

           ibq  fyearq  fqtr   fyr      ceqq  ...  dvyfirstquartile  \
4589

Add in whether firms are growth/value (44:50)

In [ ]:
table2019to2022['growth'] = False
table2019to2022['value'] = False
for row in table2019to2022.index:
    try:
        if (table2019to2022['btmfirstthird'][row] and not table2019to2022['dvyfourthquartile'][row]):
            table2019to2022.at[row, 'growth'] = True
    except:
        continue
    try:
        if (table2019to2022['btmlastthird'][row] and table2019to2022['dvyfourthquartile'][row]):
            table2019to2022.at[row, 'value'] = True
    except:
        continue

Add SRW sales/earnings surprise (46:10)

In [ ]:
for col in ['saleq', 'saleq_4', 'ibq', 'ibq_4']:
    table2019to2022[col] = pd.to_numeric(table2019to2022[col], errors='coerce')

table2019to2022['srwsalessurprise'] = np.nan
table2019to2022['srwearningssurprise'] = np.nan

table2019to2022['srwsalessurprise'] = (table2019to2022['saleq'] - table2019to2022['saleq_4']) / table2019to2022['saleq_4']
table2019to2022['srwearningssurprise'] = (table2019to2022['ibq'] - table2019to2022['ibq_4']) / table2019to2022['ibq_4']

Add RW sales/earnings surprise (47:17)

In [ ]:
for col in ['saleq', 'saleq_1', 'ibq', 'ibq_1']:
    table2019to2022[col] = pd.to_numeric(table2019to2022[col], errors='coerce')

table2019to2022['rwsalessurprise'] = np.nan
table2019to2022['rwearningssurprise'] = np.nan

table2019to2022['rwsalessurprise'] = (table2019to2022['saleq'] - table2019to2022['saleq_1']) / table2019to2022['saleq_1']
table2019to2022['rwearningssurprise'] = (table2019to2022['ibq'] - table2019to2022['ibq_1']) / table2019to2022['ibq_1']

Calculate the quarterly analyst sales/earnings surprises (48:07)

In [ ]:
file = "/content/gdrive/MyDrive/FIMA/IBESSurpriseHistory.csv"
ibessurprisehistory = pd.read_csv(file)
ibessurprisehistory['datadate'] = datetime(2010, 1, 1)
ibessurprisehistory['datadate'] = pd.to_datetime(ibessurprisehistory['datadate'], format="%Y%m%d")
ibessurprisehistory['PMON'] = pd.to_numeric(ibessurprisehistory['PMON'])
ibessurprisehistory['PYEAR'] = pd.to_numeric(ibessurprisehistory['PYEAR'])
ibessurprisehistory = ibessurprisehistory[ibessurprisehistory['PYEAR'] >= 2020]

for row in ibessurprisehistory.index:
    try:
        if ibessurprisehistory['PMON'][row] in [1, 3, 5, 7, 8, 10, 12]:
            ibessurprisehistory.at[row, 'datadate'] = datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 31)
        elif ibessurprisehistory['PMON'][row] in [4, 6, 9, 11]:
            ibessurprisehistory.at[row, 'datadate'] = datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 30)
        elif ibessurprisehistory['PMON'][row] == 2:
            if ibessurprisehistory['PYEAR'][row] == 2020:
                ibessurprisehistory.at[row, 'datadate'] = datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 29)
            elif ibessurprisehistory['PYEAR'][row] > 2020:
                ibessurprisehistory.at[row, 'datadate'] = datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 28)
    except:
        continue

Merge in analyst quarterly sales and income surprises (50:23)

In [ ]:
# Select desired columns in the specified order
new_columns = ['TICKER', 'datadate', 'rdq', 'three_day_return_if_afterhours', 'srwsalessurprise', 'srwearningssurprise',
                'rwsalessurprise', 'rwearningssurprise', 'growth', 'value', 'book_to_market_timet', 'market_value_of_equity', 'saleq']
table2019to2022 = table2019to2022[new_columns]
table2019to2022['datadate'] = pd.to_datetime(table2019to2022['datadate'], format="%Y-%m-%d")

# Drop rows where 'three_day_return_ifafterhours' is missing
table2019to2022.dropna(subset=['three_day_return_if_afterhours'], inplace=True)

#ADD IN THE qtrepssuescore
#filter ibessurprisehistory
truncated = ibessurprisehistory[(ibessurprisehistory['FISCALP'] == 'QTR') & (ibessurprisehistory['MEASURE'] == 'EPS')]
# Merge table2019to2022 with ibessurprisehistory
table2019to2022 = pd.merge(table2019to2022, truncated[['OFTIC', 'datadate', 'suescore', 'anndats']],
                     how='left',
                     left_on=['TICKER', 'datadate'],
                     right_on=['OFTIC', 'datadate'])
# Rename the columns
table2019to2022.rename(columns={'suescore': 'qtrepssuescore', 'anndats': 'qtrepsanndats'}, inplace=True)
#drop OFTIC
table2019to2022.drop('OFTIC', axis=1, inplace=True)

#ADD IN THE qtrsalessuescore
#filter ibessurprisehistory
truncated = ibessurprisehistory[(ibessurprisehistory['FISCALP'] == 'QTR') & (ibessurprisehistory['MEASURE'] == 'SAL')]
# Merge table2019to2022 with ibessurprisehistory
table2019to2022 = pd.merge(table2019to2022, truncated[['OFTIC', 'datadate', 'anndats', 'suescore']],
                     how='left',
                     left_on=['TICKER', 'datadate'],
                     right_on=['OFTIC', 'datadate'])
# Rename the columns
table2019to2022.rename(columns={'suescore': 'qtrsalessuescore', 'anndats': 'qtrsalesanndats'}, inplace=True)
#drop OFTIC
table2019to2022.drop('OFTIC', axis=1, inplace=True)

#drop rows w/o sue scores
table2019to2022.dropna(subset=['qtrepssuescore', 'qtrsalessuescore'], inplace=True)

<ipython-input-26-37a5cd12e8bf>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table2019to2022['datadate'] = pd.to_datetime(table2019to2022['datadate'], format="%Y-%m-%d")
<ipython-input-26-37a5cd12e8bf>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table2019to2022.dropna(subset=['three_day_return_if_afterhours'], inplace=True)


Drop all inf/na values (51:47)

In [ ]:
table2019to2022.replace([np.inf, -np.inf], np.nan, inplace=True)
table2019to2022 = table2019to2022.dropna()

Next, calculate summary statistics for all firms (a bit before 56:04)

In [ ]:
#First, we want to calculate the number of unique firms.
#Convert 'datadate' column to datetime if it's not already
table2019to2022['datadate'] = pd.to_datetime(table2019to2022['datadate'])
#Filter the subset based on 'datadate'
table2020to2022 = table2019to2022[table2019to2022['datadate'] >= datetime(2020, 1, 1)]
#Count the number of unique values in 'TICKER' column
totalnumberofuniquefirms = table2020to2022['TICKER'].nunique()
print("Number of Unique Firms:", totalnumberofuniquefirms)
print()

colswlabels = {
    'srwearningssurprise': "SRW Earnings Surprise",
    'srwsalessurprise': "SRW Sales Surprise",
    'rwearningssurprise': "RW Earnings Surprise",
    'rwsalessurprise': "RW Sales Surprise",
    'three_day_return_if_afterhours': "Three Day Return",
    'qtrepssuescore': "Quarterly Analyst Earnings Surprise",
    'qtrsalessuescore': "Quarterly Analyst Sales Surprise",
    'book_to_market_timet': "Book to Market",
    'saleq': "Sales",
    'market_value_of_equity': "Market Value of Equity"
}

for col in colswlabels:
    print(colswlabels[col], "Min:", table2020to2022[col].min())
    print(colswlabels[col], "1st Percentile:", table2020to2022[col].quantile(0.01))
    #print(colswlabels[col], "10th Percentile:", table2020to2022[col].quantile(0.10))
    print(colswlabels[col], "25th Percentile:", table2020to2022[col].quantile(0.25))
    print(colswlabels[col], "50th Percentile:", table2020to2022[col].quantile(0.50))
    print(colswlabels[col], "75th Percentile:", table2020to2022[col].quantile(0.75))
    print(colswlabels[col], "90th Percentile:", table2020to2022[col].quantile(0.90))
    print(colswlabels[col], "95th Percentile:", table2020to2022[col].quantile(0.95))
    print(colswlabels[col], "99th Percentile:", table2020to2022[col].quantile(0.99))
    print(colswlabels[col], "Mean:", table2020to2022[col].mean())
    print(colswlabels[col], "Max:", table2020to2022[col].max())
    print("Number of Observations:", len(table2020to2022[col]))
    print()

<ipython-input-28-ab8b1e46538a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table2019to2022['datadate'] = pd.to_datetime(table2019to2022['datadate'])


Number of Unique Firms: 3535

SRW Earnings Surprise Min: -2288.75
SRW Earnings Surprise 1st Percentile: -25.374804035285287
SRW Earnings Surprise 25th Percentile: -0.81083060563678
SRW Earnings Surprise 50th Percentile: -0.029920743923854874
SRW Earnings Surprise 75th Percentile: 0.588333442513034
SRW Earnings Surprise 90th Percentile: 2.0760881225311825
SRW Earnings Surprise 95th Percentile: 4.712991432583124
SRW Earnings Surprise 99th Percentile: 24.787863825363775
SRW Earnings Surprise Mean: -0.0881373205612824
SRW Earnings Surprise Max: 2743.3999999999996
Number of Observations: 21012

SRW Sales Surprise Min: -74.08696279203923
SRW Sales Surprise 1st Percentile: -0.9116035966241834
SRW Sales Surprise 25th Percentile: -0.0584890580055599
SRW Sales Surprise 50th Percentile: 0.07980079125465712
SRW Sales Surprise 75th Percentile: 0.2829429415931134
SRW Sales Surprise 90th Percentile: 0.7037406963064243
SRW Sales Surprise 95th Percentile: 1.2785691123958933
SRW Sales Surprise 99th Perc

Next, summary stats for all value firms (56:18)

In [ ]:
#Filter out value firms.
value_firms = table2020to2022[table2020to2022['value']]
#Count the number of unique values in 'TICKER' column
totalnumberofuniquefirms = value_firms['TICKER'].nunique()
print("Table 1 Summary Statistics for Value Firms")
print("Number of Unique Firms:", totalnumberofuniquefirms)
print()

colswlabels = {
    'srwearningssurprise': "SRW Earnings Surprise",
    'srwsalessurprise': "SRW Sales Surprise",
    'rwearningssurprise': "RW Earnings Surprise",
    'rwsalessurprise': "RW Sales Surprise",
    'three_day_return_if_afterhours': "Three Day Return",
    'qtrepssuescore': "Quarterly Analyst Earnings Surprise",
    'qtrsalessuescore': "Quarterly Analyst Sales Surprise",
    'book_to_market_timet': "Book to Market",
    'saleq': "Sales",
    'market_value_of_equity': "Market Value of Equity"
}

for col in colswlabels:
    print(colswlabels[col], "Min:", value_firms[col].min())
    print(colswlabels[col], "1st Percentile:", value_firms[col].quantile(0.01))
    #print(colswlabels[col], "10th Percentile:", value_firms[col].quantile(0.10))
    print(colswlabels[col], "25th Percentile:", value_firms[col].quantile(0.25))
    print(colswlabels[col], "50th Percentile:", value_firms[col].quantile(0.50))
    print(colswlabels[col], "75th Percentile:", value_firms[col].quantile(0.75))
    print(colswlabels[col], "90th Percentile:", value_firms[col].quantile(0.90))
    print(colswlabels[col], "95th Percentile:", value_firms[col].quantile(0.95))
    print(colswlabels[col], "99th Percentile:", value_firms[col].quantile(0.99))
    print(colswlabels[col], "Mean:", value_firms[col].mean())
    print(colswlabels[col], "Max:", value_firms[col].max())
    print("Number of Observations:", len(value_firms[col]))
    print()

Table 1 Summary Statistics for Value Firms
Number of Unique Firms: 696

SRW Earnings Surprise Min: -1809.2727272727273
SRW Earnings Surprise 1st Percentile: -30.121089471013356
SRW Earnings Surprise 25th Percentile: -0.9225512528473804
SRW Earnings Surprise 50th Percentile: -0.07951833369494474
SRW Earnings Surprise 75th Percentile: 0.4894179894179894
SRW Earnings Surprise 90th Percentile: 1.8107544732314618
SRW Earnings Surprise 95th Percentile: 4.1779381957703485
SRW Earnings Surprise 99th Percentile: 25.614366700899332
SRW Earnings Surprise Mean: 0.10601809010225918
SRW Earnings Surprise Max: 1363.5714285714287
Number of Observations: 3149

SRW Sales Surprise Min: -29.114754098360656
SRW Sales Surprise 1st Percentile: -0.9306386364328203
SRW Sales Surprise 25th Percentile: -0.10699588477366255
SRW Sales Surprise 50th Percentile: -0.007200411452082976
SRW Sales Surprise 75th Percentile: 0.10593797334720555
SRW Sales Surprise 90th Percentile: 0.3425261144065984
SRW Sales Surprise 95th

Next, summary stats for all growth firms (57:11)

In [ ]:
#Filter out growth firms.
growth_firms = table2020to2022[table2020to2022['growth']]
#Count the number of unique values in 'TICKER' column
totalnumberofuniquefirms = growth_firms['TICKER'].nunique()
print("Table 1 Summary Statistics for Growth Firms")
print("Number of Unique Firms:", totalnumberofuniquefirms)
print()

colswlabels = {
    'srwearningssurprise': "SRW Earnings Surprise",
    'srwsalessurprise': "SRW Sales Surprise",
    'rwearningssurprise': "RW Earnings Surprise",
    'rwsalessurprise': "RW Sales Surprise",
    'three_day_return_if_afterhours': "Three Day Return",
    'qtrepssuescore': "Quarterly Analyst Earnings Surprise",
    'qtrsalessuescore': "Quarterly Analyst Sales Surprise",
    'book_to_market_timet': "Book to Market",
    'saleq': "Sales",
    'market_value_of_equity': "Market Value of Equity"
}

for col in colswlabels:
    print(colswlabels[col], "Min:", growth_firms[col].min())
    print(colswlabels[col], "1st Percentile:", growth_firms[col].quantile(0.01))
    #print(colswlabels[col], "10th Percentile:", growth_firms[col].quantile(0.10))
    print(colswlabels[col], "25th Percentile:", growth_firms[col].quantile(0.25))
    print(colswlabels[col], "50th Percentile:", growth_firms[col].quantile(0.50))
    print(colswlabels[col], "75th Percentile:", growth_firms[col].quantile(0.75))
    print(colswlabels[col], "90th Percentile:", growth_firms[col].quantile(0.90))
    print(colswlabels[col], "95th Percentile:", growth_firms[col].quantile(0.95))
    print(colswlabels[col], "99th Percentile:", growth_firms[col].quantile(0.99))
    print(colswlabels[col], "Mean:", growth_firms[col].mean())
    print(colswlabels[col], "Max:", growth_firms[col].max())
    print("Number of Observations:", len(growth_firms[col]))
    print()

Table 1 Summary Statistics for Growth Firms
Number of Unique Firms: 1585

SRW Earnings Surprise Min: -2288.75
SRW Earnings Surprise 1st Percentile: -17.830840773809523
SRW Earnings Surprise 25th Percentile: -0.49983016304347827
SRW Earnings Surprise 50th Percentile: 0.0766511250313763
SRW Earnings Surprise 75th Percentile: 0.6837424311253344
SRW Earnings Surprise 90th Percentile: 2.082045951276724
SRW Earnings Surprise 95th Percentile: 4.474128107596249
SRW Earnings Surprise 99th Percentile: 21.530928247627966
SRW Earnings Surprise Mean: 0.3515722819351633
SRW Earnings Surprise Max: 2743.3999999999996
Number of Observations: 6720

SRW Sales Surprise Min: -18.66766467065868
SRW Sales Surprise 1st Percentile: -0.9009065305620464
SRW Sales Surprise 25th Percentile: 0.015361507143127387
SRW Sales Surprise 50th Percentile: 0.16837061715951812
SRW Sales Surprise 75th Percentile: 0.41597557202283764
SRW Sales Surprise 90th Percentile: 0.9641078677393331
SRW Sales Surprise 95th Percentile: 1.9

Winsorize the variables (57:01)

In [ ]:
#winsorize qtrepssuescore
table2020to2022['percentile'] = table2020to2022.groupby(['datadate'])['qtrepssuescore'].rank(pct=True)
table2020to2022['qtrepssuescorefirstpercentile'] = 0
table2020to2022['qtrepssuescoreninetyninthpercentile'] = 0
for row in table2020to2022.index:
    try:
        if table2020to2022['percentile'][row] <= .01:
            table2020to2022.loc['qtrepssuescorefirstpercentile'][row] = 1
        elif table2020to2022['percentile'][row] >= .99:
            table2020to2022.loc['qtrepssuescoreninetyninthpercentile'][row] = 1
    except:
        continue

#winsorize qtrsalessuescore
table2020to2022['percentile'] = table2020to2022.groupby(['datadate'])['qtrsalessuescore'].rank(pct=True)
table2020to2022['qtrsalessuescorefirstpercentile'] = 0
table2020to2022['qtrsalessuescoreninetyninthpercentile'] = 0
for row in table2020to2022.index:
    try:
        if table2020to2022['percentile'][row] <= .01:
            table2020to2022.loc['qtrsalessuescorefirstpercentile'][row] = 1
        elif table2020to2022['percentile'][row] >= .99:
            table2020to2022.loc['qtrsalessuescoreninetyninthpercentile'][row] = 1
    except:
        continue

#winsorize threedayreturn
table2020to2022['percentile'] = table2020to2022.groupby(['datadate'])['three_day_return_if_afterhours'].rank(pct=True)
table2020to2022['threedayfirstpercentile'] = 0
table2020to2022['threedayninetyninthpercentile'] = 0
for row in table2020to2022.index:
    try:
        if table2020to2022['percentile'][row] <= .01:
            table2020to2022.loc['threedayfirstpercentile'][row] = 1
        elif table2020to2022['percentile'][row] >= .99:
            table2020to2022.loc['threedayninetyninthpercentile'][row] = 1
    except:
        continue

#drop extrema
#define the columns to check for value equality
columns_to_check = ['qtrepssuescorefirstpercentile', 'qtrepssuescoreninetyninthpercentile',
                    'qtrsalessuescorefirstpercentile', 'qtrsalessuescoreninetyninthpercentile',
                    'threedayfirstpercentile', 'threedayninetyninthpercentile']

#filter df to exclude rows where any of the columns equal 1
table2020to2022 = table2020to2022[~table2020to2022[columns_to_check].eq(1).any(axis=1)]

More Winsorization

In [ ]:
#winsorize srwearningssurprise
table2020to2022['percentile'] = table2020to2022.groupby(['datadate'])['srwearningssurprise'].rank(pct=True)
table2020to2022['srwearningssurprisefirstpercentile'] = 0
table2020to2022['srwearningssurpriseninetyninthpercentile'] = 0
for row in table2020to2022.index:
    try:
        if table2020to2022['percentile'][row] <= .01:
            table2020to2022.loc['srwearningssurprisefirstpercentile'][row] = 1
        elif table2020to2022['percentile'][row] >= .99:
            table2020to2022.loc['srwearningssurpriseninetyninthpercentile'][row] = 1
    except:
        continue

#winsorize srwsalessurprise
table2020to2022['percentile'] = table2020to2022.groupby(['datadate'])['srwsalessurprise'].rank(pct=True)
table2020to2022['srwsalessurprisefirstpercentile'] = 0
table2020to2022['srwsalessurpriseninetyninthpercentile'] = 0
for row in table2020to2022.index:
    try:
        if table2020to2022['percentile'][row] <= .01:
            table2020to2022.loc['srwsalessurprisefirstpercentile'][row] = 1
        elif table2020to2022['percentile'][row] >= .99:
            table2020to2022.loc['srwsalessurpriseninetyninthpercentile'][row] = 1
    except:
        continue

#winsorize rwearningssurprise
table2020to2022['percentile'] = table2020to2022.groupby(['datadate'])['rwearningssurprise'].rank(pct=True)
table2020to2022['rwearningssurprisefirstpercentile'] = 0
table2020to2022['rwearningssurpriseninetyninthpercentile'] = 0
for row in table2020to2022.index:
    try:
        if table2020to2022['percentile'][row] <= .01:
            table2020to2022.loc['rwearningssurprisefirstpercentile'][row] = 1
        elif table2020to2022['percentile'][row] >= .99:
            table2020to2022.loc['rwearningssurpriseninetyninthpercentile'][row] = 1
    except:
        continue

#winsorize rwsalessurprise
table2020to2022['percentile'] = table2020to2022.groupby(['datadate'])['rwsalessurprise'].rank(pct=True)
table2020to2022['rwsalessurprisefirstpercentile'] = 0
table2020to2022['rwsalessurpriseninetyninthpercentile'] = 0
for row in table2020to2022.index:
    try:
        if table2020to2022['percentile'][row] <= .01:
            table2020to2022.loc['rwsalessurprisefirstpercentile'][row] = 1
        elif table2020to2022['percentile'][row] >= .99:
            table2020to2022.loc['rwsalessurpriseninetyninthpercentile'][row] = 1
    except:
        continue

#drop extrema
#define the columns to check for value equality
columns_to_check = ['srwearningssurprisefirstpercentile', 'srwearningssurpriseninetyninthpercentile',
                    'srwsalessurprisefirstpercentile', 'srwsalessurpriseninetyninthpercentile',
                    'rwearningssurprisefirstpercentile', 'rwearningssurpriseninetyninthpercentile',
                    'rwsalessurprisefirstpercentile', 'rwsalessurpriseninetyninthpercentile']

#filter df to exclude rows where any of the columns equal 1
table2020to2022 = table2020to2022[~table2020to2022[columns_to_check].eq(1).any(axis=1)]

Regress three day return on analyst qearnings and sales (1:01:00)

In [ ]:
# Specify the formula for the regression equation
formula = "three_day_return_if_afterhours ~ qtrsalessuescore + qtrepssuescore"

# Create the OLS model and fit it to the data
threedayreturnmodeltable2 = sm.OLS.from_formula(formula, data=table2020to2022).fit()

# Summarize the model
threedayreturnmodeltable2_summary = threedayreturnmodeltable2.summary()
print(threedayreturnmodeltable2_summary)


                                  OLS Regression Results                                  
Dep. Variable:     three_day_return_if_afterhours   R-squared:                       0.002
Model:                                        OLS   Adj. R-squared:                  0.002
Method:                             Least Squares   F-statistic:                     18.59
Date:                            Sun, 23 Jul 2023   Prob (F-statistic):           8.55e-09
Time:                                    01:07:23   Log-Likelihood:                 20690.
No. Observations:                           21012   AIC:                        -4.137e+04
Df Residuals:                               21009   BIC:                        -4.135e+04
Df Model:                                       2                                         
Covariance Type:                        nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----

Table 2 Panel A

In [ ]:
# Specify the formula for the regression equation
formula2_a_1 = "three_day_return_if_afterhours ~ qtrepssuescore"
formula2_a_2 = "three_day_return_if_afterhours ~ qtrsalessuescore"

# Create the OLS model and fit it to the data
threedayreturnmodeltable2_a_1 = sm.OLS.from_formula(formula2_a_1, data=table2020to2022).fit()
threedayreturnmodeltable2_a_2 = sm.OLS.from_formula(formula2_a_2, data=table2020to2022).fit()

# Summarize the model
threedayreturnmodeltable2_a_1_summary = threedayreturnmodeltable2_a_1.summary()
print(threedayreturnmodeltable2_a_1_summary)
threedayreturnmodeltable2_a_2_summary = threedayreturnmodeltable2_a_2.summary()
print(threedayreturnmodeltable2_a_2_summary)

                                  OLS Regression Results                                  
Dep. Variable:     three_day_return_if_afterhours   R-squared:                       0.001
Model:                                        OLS   Adj. R-squared:                  0.001
Method:                             Least Squares   F-statistic:                     28.51
Date:                            Sun, 23 Jul 2023   Prob (F-statistic):           9.43e-08
Time:                                    01:07:23   Log-Likelihood:                 20686.
No. Observations:                           21012   AIC:                        -4.137e+04
Df Residuals:                               21010   BIC:                        -4.135e+04
Df Model:                                       1                                         
Covariance Type:                        nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
-------

Table 2 Panel B

In [ ]:
# Specify the formula for the regression equation
formula2_b_1 = "three_day_return_if_afterhours ~ srwearningssurprise"
formula2_b_2 = "three_day_return_if_afterhours ~ srwsalessurprise"
formula2_b_3= "three_day_return_if_afterhours ~ rwearningssurprise"
formula2_b_4 = "three_day_return_if_afterhours ~ rwsalessurprise"

# Create the OLS model and fit it to the data
threedayreturnmodeltable2_b_1 = sm.OLS.from_formula(formula2_b_1, data=table2020to2022).fit()
threedayreturnmodeltable2_b_2 = sm.OLS.from_formula(formula2_b_2, data=table2020to2022).fit()
threedayreturnmodeltable2_b_3 = sm.OLS.from_formula(formula2_b_3, data=table2020to2022).fit()
threedayreturnmodeltable2_b_4 = sm.OLS.from_formula(formula2_b_4, data=table2020to2022).fit()

# Summarize the model
threedayreturnmodeltable2_b_1_summary = threedayreturnmodeltable2_b_1.summary()
print(threedayreturnmodeltable2_b_1_summary)
threedayreturnmodeltable2_b_2_summary = threedayreturnmodeltable2_b_2.summary()
print(threedayreturnmodeltable2_b_2_summary)
threedayreturnmodeltable2_b_3_summary = threedayreturnmodeltable2_b_3.summary()
print(threedayreturnmodeltable2_b_3_summary)
threedayreturnmodeltable2_b_4_summary = threedayreturnmodeltable2_b_4.summary()
print(threedayreturnmodeltable2_b_4_summary)

                                  OLS Regression Results                                  
Dep. Variable:     three_day_return_if_afterhours   R-squared:                       0.000
Model:                                        OLS   Adj. R-squared:                 -0.000
Method:                             Least Squares   F-statistic:                    0.4759
Date:                            Sun, 23 Jul 2023   Prob (F-statistic):              0.490
Time:                                    01:07:23   Log-Likelihood:                 20672.
No. Observations:                           21012   AIC:                        -4.134e+04
Df Residuals:                               21010   BIC:                        -4.132e+04
Df Model:                                       1                                         
Covariance Type:                        nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
--

Regression with value firm, growth dummies etc. (1:03:41)

In [ ]:
table2020to2022['valuedummy'] = table2020to2022['value'].astype(int)
table2020to2022['qtrsalessuescoreXvaluedummy'] = table2020to2022['qtrsalessuescore'] * table2020to2022['valuedummy']
table2020to2022['qtrepssuescoreXvaluedummy'] = table2020to2022['qtrepssuescore'] * table2020to2022['valuedummy']

# Specify the formula for the regression equation
formula_3_1 = "three_day_return_if_afterhours ~ valuedummy + qtrepssuescore + qtrepssuescoreXvaluedummy"
formula_3_2 = "three_day_return_if_afterhours ~ valuedummy + qtrsalessuescore + qtrsalessuescoreXvaluedummy"

# Create the OLS model and fit it to the data
threedayreturnmodeltable3_1 = sm.OLS.from_formula(formula_3_1, data=table2020to2022).fit()
threedayreturnmodeltable3_2 = sm.OLS.from_formula(formula_3_2, data=table2020to2022).fit()

# Summarize the model
threedayreturnmodeltable3_1_summary = threedayreturnmodeltable3_1.summary()
threedayreturnmodeltable3_2_summary = threedayreturnmodeltable3_2.summary()
print(threedayreturnmodeltable3_1_summary)
print(threedayreturnmodeltable3_2_summary)


                                  OLS Regression Results                                  
Dep. Variable:     three_day_return_if_afterhours   R-squared:                       0.007
Model:                                        OLS   Adj. R-squared:                  0.007
Method:                             Least Squares   F-statistic:                     50.51
Date:                            Sun, 23 Jul 2023   Prob (F-statistic):           1.62e-32
Time:                                    01:07:23   Log-Likelihood:                 20747.
No. Observations:                           21012   AIC:                        -4.149e+04
Df Residuals:                               21008   BIC:                        -4.145e+04
Df Model:                                       3                                         
Covariance Type:                        nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.9

For ad hoc additions

In [ ]:
#table2019to2022.drop("dvyfourthquartile", axis=1, inplace=True)


Printer

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
print(table2020to2022[table2020to2022['TICKER'].isin(['KO', 'PEP'])].head(50))
#print(ibessurprisehistory[ibessurprisehistory['TICKER'].isin(['KO', 'PEP'])].head(50))

      TICKER   datadate                  rdq  three_day_return_if_afterhours  \
15771     KO 2020-03-31  2020-04-21 00:00:00                        0.001102   
15772     KO 2020-06-30  2020-07-21 00:00:00                        0.027332   
15773     KO 2020-09-30  2020-10-22 00:00:00                       -0.015391   
15774     KO 2020-12-31  2021-02-10 00:00:00                        0.013507   
15775     KO 2021-03-31  2021-04-19 00:00:00                        0.008148   
15776     KO 2021-06-30  2021-07-21 00:00:00                        0.009019   
15777     KO 2021-09-30  2021-10-27 00:00:00                        0.011708   
15778     KO 2021-12-31  2022-02-10 00:00:00                       -0.007657   
21601    PEP 2020-03-31  2020-04-28 00:00:00                       -0.045334   
21602    PEP 2020-06-30  2020-07-13 00:00:00                       -0.007635   
21603    PEP 2020-09-30  2020-10-01 00:00:00                       -0.036222   
21604    PEP 2020-12-31  2021-02-11 00:0